In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf,explode
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/20 16:39:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/20 16:39:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/Product')

In [4]:
sample_data = data.select('value').take(5)

In [5]:
data_sample = [loads(item.value) for item in sample_data]

In [6]:
schema = StructType([ 
    StructField("id",IntegerType(),True), 
    StructField("master_id",IntegerType(),True), 
    StructField("sku",StringType(),True), 
    StructField("name",StringType(),True),
    StructField("short_description",StringType(),True), 
    StructField("price",IntegerType(),True),
    StructField("list_price",IntegerType(),True),
    StructField('original_price', IntegerType(),True),
    StructField('discount', IntegerType(),True),
    StructField('discount_rate', FloatType(),True),
    
    StructField("rating_average",FloatType(),True), 
    StructField("review_count",IntegerType(),True), 
    StructField("productset_group_name",StringType(),True), 
    StructField("all_time_quantity_sold",IntegerType(),True),
    
    StructField("description",StringType(),True), 
    StructField("current_seller",MapType(StringType(),StringType()),True),
    StructField("other_sellers",ArrayType(MapType(StringType(),StringType())),True),
    StructField("breadcrumbs",ArrayType(MapType(StringType(),StringType())),True),
    StructField("specifications",ArrayType(StructType(
                    [
                        StructField("name", StringType()),
                        StructField("attributes",ArrayType(MapType(StringType(),StringType())),True),
                    ]
                )),True),

    StructField('return_and_exchange_policy', StringType(),True)
])

In [7]:
df = data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")

In [8]:
df.createOrReplaceTempView('Product')

In [9]:
def extrct_other_seller(arr,current_id):
    list_id = []
    list_id.append(current_id)
    for item in arr:
        list_id.append(item['id'])
    return list_id

In [10]:
spark.udf.register('extrct_other_seller',extrct_other_seller,ArrayType(StringType()))

<function __main__.extrct_other_seller(arr, current_id)>

In [11]:
# df_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')
# df_shop.createOrReplaceTempView("shop")

In [12]:
# spark.sql("""
#     select * from shop
#     where id = 2152
# """).collect()

In [13]:
# spark.sql("""
#     select *, explode(extrct_other_seller(other_sellers,current_seller.id)) sellers from Product
#     where id = 20557588
# """).collect()

In [14]:
def parserAtt(specifications):
    result = ""
    try:
        for s in specifications:
            for a in s.attributes:
                result += a['value']
        result = cleanText(result)
    except:
        return ""
    return result

In [15]:
spark.udf.register('parserAtt',parserAtt,StringType())

<function __main__.parserAtt(specifications)>

In [16]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 
def cleanText(str_raw):
    # remove tags html
    str_raw = re.sub(CLEANR, ' ', str_raw)

    # remove special character
    str_raw = re.sub('\W+', ' ', str_raw)
    
    # remove number
    str_raw = re.sub("[0-9]+", "", str_raw)
    
    # remove space
    cleantext = re.sub(" +", " ", str_raw)
    return cleantext.lower()

In [17]:
spark.udf.register("cleanText", cleanText,StringType())

<function __main__.cleanText(str_raw)>

In [34]:
# product_clean = spark.sql("""
#         select id,master_id,sku,price,list_price,original_price,discount,discount_rate,
#         rating_average,review_count,productset_group_name,all_time_quantity_sold,
#         cleanText(name) name,cleanText(description) description,parserAtt(specifications) specifications,
#         current_seller.id seller_id,current_seller.name seller_name,current_seller.store_id seller_store_id,
#         cast(current_seller.product_id as int) product_id,
#         breadcrumbs[0].name category_name,breadcrumbs[0].category_id category_id,
#         explode(extrct_other_seller(other_sellers,current_seller.id)) sellers
#         from Product
# """)

In [18]:
product_clean = spark.sql("""
        select id,master_id,sku,price,list_price,original_price,discount,discount_rate,
        rating_average,review_count,productset_group_name,all_time_quantity_sold,
        cleanText(name) name,cleanText(description) description,parserAtt(specifications) specifications,
        breadcrumbs[0].name category_name,breadcrumbs[0].category_id category_id
        from Product
""")

In [20]:
product_clean.write.partitionBy("category_id").mode('append').parquet('hdfs://namenode:9000/TikiCleaned/Product')

In [21]:
product_clean.count()

46721

In [30]:
product_shop_info = spark.sql("""
        select id,explode(extrct_other_seller(other_sellers,current_seller.id)) sellers,breadcrumbs[0].category_id category_id
        from Product
""")

In [32]:
product_shop_info.write.partitionBy("category_id").mode('append').parquet('hdfs://namenode:9000/TikiCleaned/Product_Shop')

In [33]:
product_shop_info.count()

127069